In [ ]:
import os, langchain, faiss, time
from uuid import uuid4
langchain.verbose = True
langchain.debug = True

from collections import deque
from pathlib import Path
from elasticsearch import Elasticsearch
from langchain.schema import format_document
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS, Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_mistralai import MistralAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader, UnstructuredPowerPointLoader, UnstructuredWordDocumentLoader

from prompt import CHAT_PROMPT, DOCUMENT_PROMPT
from config import load_config_data

huggingface_base_url = load_config_data["HUGGINGFACE_NOVITA"]
huggingface_api_key = load_config_data["HUGGINGFACE_API_TOKEN"]
chat_model_llama = load_config_data["LLAMA_3_3_70B"]
chat_model_llama_1 = load_config_data["LLAMA_3_3_70B_1"]

mistral_base_url = load_config_data["MISTRAL_BASE_URL"]
mistral_api_key = load_config_data["MISTRAL_API_KEY"]
mistral_embedding_model = load_config_data["MISTRAL_EMBEDDING"]

vector_db_path = Path(load_config_data["VECTOR_DB_PATH"])

db_type = 'faiss'
db_name = 'Test_index_1'
index_name = 'Test Index 1'

In [ ]:
mistral_embedding = MistralAIEmbeddings(api_key=mistral_api_key, model=mistral_embedding_model)
huggingface_embedding = HuggingFaceEmbeddings(model_name="/home/laxmidhar/Coding Stuff/AI Models/multilingual-e5-large-instruct")
gpt_client = ChatOpenAI(api_key=huggingface_api_key, model=chat_model_llama_1, base_url=huggingface_base_url)

In [ ]:
def load_db(db, folder_path, db_name, embedding):
    return db.load_local(os.path.join(folder_path, db_name), 
                         embeddings=embedding, 
                         allow_dangerous_deserialization=True)

In [ ]:
def vector_DB(db_type, db_name, index_name, langchain_embedding):
    '''Please provide DB type (faiss or chroma), DB name and index name'''
    if db_type.lower() == 'faiss':
        index_id = faiss.IndexFlatL2(len(langchain_embedding.embed_query(index_name)))
        db = FAISS(embedding_function=langchain_embedding,
                    docstore=InMemoryDocstore(), 
                    index_to_docstore_id={}, 
                    index=index_id, 
                    distance_strategy=DistanceStrategy.COSINE)
        if os.path.isdir(os.path.join(vector_db_path, db_name)):
            faiss_db = load_db(db, vector_db_path, db_name, langchain_embedding)
            print('Existing DB found. \nLoaded successfully')
            print(f"Total {len(faiss_db.index_to_docstore_id)} documents exist in the DB")
            return faiss_db
        else:
            db.save_local(os.path.join(vector_db_path, db_name))
            faiss_db = load_db(db, vector_db_path, db_name, langchain_embedding)
            print(f"Total {len(faiss_db.index_to_docstore_id)} documents exist in the DB")
            print('New FAISS DB created')
            return faiss_db
    elif db_type.lower() == 'chroma':
        chroma_DB = Chroma(persist_directory=os.path.join(vector_db_path, db_name), 
                           embedding_function=langchain_embedding, 
                           collection_name=index_name)
        print('You are using Chroma DB')
        print(f"Total {len(chroma_DB.get()['documents'])} documents exist in the DB")
        return 'Presently Chroma DB is not available'
    elif db_type.lower() == 'elastic':
        es = Elasticsearch("http://localhost:9200")
        elastic_vector_store = ElasticsearchStore(es_connection = es, 
                                                  index_name = index_name, 
                                                  embedding = langchain_embedding, 
                                                  distance_strategy=DistanceStrategy.COSINE, 
                                                  strategy=DenseVectorStrategy())
        print(f"Total {es.count(index=index_name)} documents exist in the DB")
        # es.indices.delete(index=index_name)
        return elastic_vector_store
    else:
        return f"Invalid db type {db_type}. Please provide a valid db type"
vector_db = vector_DB(db_type, db_name, index_name, huggingface_embedding)

In [ ]:
def file_extraction(file:str):
    if os.path.basename(file).lower().endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(file, mode="elements", strategy="fast") 
        docs = loader.load()
    elif os.path.basename(file).lower().endswith(".pdf"):
        loader = PDFPlumberLoader(file) 
        docs = loader.load()
    elif os.path.basename(file).lower().endswith(".pptx"):
        loader = UnstructuredPowerPointLoader(file, mode="elements", strategy="fast") 
        docs = loader.load()
    return docs

In [ ]:
def _combine_documents(docs, document_prompt=DOCUMENT_PROMPT, document_separator="\n\n"):
    return document_separator.join(
        f"{format_document(doc, document_prompt)}\n(Source: {os.path.basename(doc.metadata.get('source', 'Unknown'))})"
        for doc in docs
    )

In [ ]:
conversation_history = deque(maxlen=5)

def get_history():
    """
    Formats the conversation history for inclusion in the prompt.
    """
    return "\n".join(f"User: {q}\nBot: {r}" for q, r in conversation_history)

In [ ]:
def data_inject_DB(file:str) -> str:
    doc = file_extraction(file)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20, length_function = len, is_separator_regex=False)
    splitted_docs = text_splitter.split_documents(doc)
    vector_db.add_documents(documents=splitted_docs, ids=[str(uuid4()) for _ in range(len(splitted_docs))])
    print(f"Storing {len(splitted_docs)} documents")
    if db_type.lower() == 'faiss':
        vector_db.save_local(os.path.join(vector_db_path, db_name))
        return f"Total {len(vector_db.index_to_docstore_id)} number of documents stored in the DB"
    return f"Total {len(vector_db.get()['documents'])} number of document stored in the DB"

In [ ]:
def chat_response(query:str):
    retriever = vector_db.as_retriever(search_kwargs={"k": 3})
    # retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={'k': 4, 'score_threshold': 0.75, 'fetch_k': 10})#search_kwargs={"k": 5})
    docs = retriever.invoke(query)
    print(docs)
    if not docs:
        no_docs_message = "I'm sorry, but I couldn't find any relevant information in my knowledge base to answer your query. Please try rephrasing or asking something else."
        yield no_docs_message
        conversation_history.append((query, no_docs_message))
        return
    combined_context = _combine_documents(docs)
    history = get_history()
    chain = (
        {
            "context": RunnablePassthrough(), 
            "history": RunnablePassthrough(), 
            "question": RunnablePassthrough()
            }
            | CHAT_PROMPT
            | gpt_client
            | StrOutputParser()
            )
    chain_input = chain_input = {"context": combined_context, "history": history, "question": query}
    final_output = chain.stream(chain_input)
    streamed_output = ""
    for chunk in final_output:
        streamed_output += chunk
        yield chunk
    conversation_history.append((query, streamed_output))
    # final_output = chain.invoke(formatted_input)
    # conversation_history.append((query, final_output))
    # return final_output

In [ ]:
query = "Tell me something about AI"

for token in chat_response(query):
    print(token)